- Main KITTI readme (old): https://github.com/yanii/kitti-pcl/blob/master/KITTI_README.TXT  
- About Intrinsics: http://ksimek.github.io/2013/08/13/intrinsic/
- Car scheme: http://www.cvlibs.net/publications/Geiger2013IJRR.pdf
- https://avisingh599.github.io/vision/visual-odometry-full/
- Image with reprojection: https://yadi.sk/i/JAIIsbP5dHAELg
- https://github.com/cgarg92/Stereo-visual-odometry

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints
from shared.tools import find_max_clique

%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR)

In [ ]:
frame_idx = 150
c_l_img, c_r_img = dataset.get_color_images(frame_idx)
n_l_img, n_r_img = dataset.get_color_images(frame_idx+1)
Q_left = dataset.get_color_left_Q_matrix()
gt_transform = dataset._get_transform_mtrx(frame_idx)

print(gt_transform)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_l_img)
ax2.imshow(n_l_img)

In [ ]:
vo = VisualOdometry()
c_depth_frame = vo.process_depth(c_l_img, c_r_img, Q_left)
n_depth_frame = vo.process_depth(n_l_img, n_r_img, Q_left)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_depth_frame[:,:,2])
ax2.imshow(n_depth_frame[:,:,2])

In [ ]:
c_feats, n_feats = vo.get_features(c_l_img, n_l_img)

c_img_canvas = c_l_img.copy()
n_img_canvas = n_l_img.copy()

c_pnts, c_ft_idxs = vo.reproject_2d_to_3d_points(c_feats, c_depth_frame)
n_pnts, n_ft_idxs = vo.reproject_2d_to_3d_points(n_feats, n_depth_frame)

ft_idxs = c_ft_idxs & n_ft_idxs

c_pnts = c_pnts[ft_idxs]
n_pnts = n_pnts[ft_idxs]
c_feats = c_feats[ft_idxs]
n_feats = n_feats[ft_idxs]

draw_keypoints(c_img_canvas, n_img_canvas, c_feats, n_feats)

# Rendering valid features
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

In [ ]:
import networkx as nx
from networkx.algorithms.approximation.clique import max_clique

num_points = c_pnts.shape[0]
graph = nx.Graph()
graph.add_nodes_from(list(range(num_points)))

print(num_points)
# print(graph.number_of_nodes())

if c_pnts.shape[0] < 6:
    raise Exception('Too low count of points')

clique_len = 0
dist_thrs = 0.05
while clique_len < 6:
    for i in range(num_points):
        diff_1 = c_pnts[i,:] - c_pnts
        diff_2 = n_pnts[i,:] - n_pnts
        dist_1 = np.linalg.norm(diff_1, axis=1)
        dist_2 = np.linalg.norm(diff_2, axis=1)
        diff = abs(dist_2 - dist_1)
        wIdx = np.where(diff < dist_thrs)
        for i_w in wIdx[0]:  
            graph.add_edge(i, i_w)

    cliques = nx.algorithms.find_cliques(graph)
    max_clique = max_clique(graph)
    clique_len = len(max_clique)
    
print(clique_len)

c_pnts_new = c_pnts[list(max_clique)]
n_pnts_new = n_pnts[list(max_clique)]

In [ ]:
# P_left, _ = dataset.get_color_P_matrix()
# print(P_left)

С_left, _ = dataset.get_color_С_matrix()
P_left = np.hstack((С_left, np.transpose([[0, 0, 0]])))

c_pnts_2d = vo.reproject_3d_to_2d(c_pnts_new, P_left)
n_pnts_2d = vo.reproject_3d_to_2d(n_pnts_new, P_left)

c_img_canvas = c_l_img.copy()
n_img_canvas = n_l_img.copy()

# Check after clique and projection
draw_keypoints(c_img_canvas, n_img_canvas, c_pnts_2d, n_pnts_2d)

# # Rendering valid features
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

In [ ]:
from scipy.spatial.transform import Rotation as R
from scipy.optimize import least_squares

def get_transform_matrix(x):
    transform = np.eye(4)
    transform[:3,:3] = R.from_rotvec(x[:3]).as_matrix()
    transform[:3, 3] = x[3:]
    return transform
    
def from_transform_matrix(x):
    rotvec = R.from_matrix(x[:3,:3]).as_rotvec()
    transl = x[:3,3]
    return list([*rotvec, *transl])
    
def estimate_transform_2d(x, c_pnts_2d, n_pnts_2d, c_pnts_3d, n_pnts_3d, P_mtrx):
    """
        x - [rotvec, transform]
    """
    transform = get_transform_matrix(x)

    Proj_frwrd = P_mtrx @ np.linalg.inv(transform)
    Proj_bcwrd = P_mtrx @ transform
    
    n_pred_pnts_2d = vo.reproject_3d_to_2d(c_pnts_3d, Proj_frwrd)
    c_pred_pnts_2d = vo.reproject_3d_to_2d(n_pnts_3d, Proj_bcwrd)
    
    c_err = c_pnts_2d - c_pred_pnts_2d
    n_err = n_pnts_2d - n_pred_pnts_2d

    residual = np.vstack((c_err*c_err,n_err*n_err))
    return residual.flatten()

def estimate_transform_3d(x, c_pnts_3d, n_pnts_3d):
    """
        x - [rotvec, transform]
    """
    transform = get_transform_matrix(x)
    
    c_pred_pnts_3d = vo.transform_3d(transform, n_pnts_3d)
    n_pred_pnts_3d = vo.transform_3d(np.linalg.inv(transform), c_pnts_3d)
    
    c_err = c_pnts_3d - c_pred_pnts_3d
    n_err = n_pnts_3d - n_pred_pnts_3d

    residual = np.vstack((c_err*c_err,n_err*n_err))
    return residual.flatten()

In [ ]:
c_pnts_new

In [ ]:
n_pnts_new

In [ ]:
initial = from_transform_matrix(gt_transform)
print(initial)

estimate_transform_2d(
    initial,
    c_pnts_2d,
    n_pnts_2d,
    c_pnts_new,
    n_pnts_new,
    P_left
)

In [ ]:
initial = from_transform_matrix(gt_transform)
print(initial)

estimate_transform_3d(
    initial,
    c_pnts_new,
    n_pnts_new
)

In [ ]:
initial = np.zeros(6)
args = (
    c_pnts_2d,
    n_pnts_2d,
    c_pnts_new,
    n_pnts_new,
    P_left
)
optRes = least_squares(estimate_transform, initial, method='lm', max_nfev=200, args=args, verbose=2)

In [ ]:
optRes.x

In [ ]:
optRes